install ollama and instructor
```
pip install instructor
```
Ollama download instruction are available here 
https://ollama.com/download

Download the heremes pro model 
Heremes pro is latest model from NousResearch. Somebody already uploaded to ollama model library

```
ollama pull adrienbrault/nous-hermes2pro:Q8_0
```
And then you can use the model in the code below



In [51]:
from openai import OpenAI
from pydantic import BaseModel
import instructor
import json

In [3]:
# enables `response_model` in create call
client = instructor.patch(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

In [2]:
from typing import List
class QuestionAnswer(BaseModel):
    question: str
    answer: str
    citations: List[str]
QuestionAnswer.model_json_schema()

{'properties': {'question': {'title': 'Question', 'type': 'string'},
  'answer': {'title': 'Answer', 'type': 'string'},
  'citations': {'items': {'type': 'string'},
   'title': 'Citations',
   'type': 'array'}},
 'required': ['question', 'answer', 'citations'],
 'title': 'QuestionAnswer',
 'type': 'object'}

In [32]:

def ask_agent(question: str, context: str) -> QuestionAnswer:
    messages=[
            {
                "role": "system",
                "content": "You are a world class agentic framework to answer questions with correct and exact citations.",
            },
            {"role": "user", "content": f"{context}"},
            {"role": "user", "content": f"Question: {question}"},
        ]
    return client.chat.completions.create(
        model="adrienbrault/nous-hermes2pro:Q4_0",
        temperature=0.1,
        response_model=QuestionAnswer,
        messages=messages,
    )

In [44]:
from typing import List
class SelfReflection(BaseModel):
    scratchpad: str
    sub_questions: List[str]
SelfReflection.model_json_schema()

{'properties': {'scratchpad': {'title': 'Scratchpad', 'type': 'string'},
  'sub_questions': {'items': {'type': 'string'},
   'title': 'Sub Questions',
   'type': 'array'}},
 'required': ['scratchpad', 'sub_questions'],
 'title': 'SelfReflection',
 'type': 'object'}

In [58]:
from concurrent.futures import ThreadPoolExecutor

def ask_ai(question: str) -> SelfReflection:
    messages=[
            {
                "role": "system",
                "content": "You are a world class agentic framework to decompose a given question into 5 sub-questions to help collect evidences for final answer. You must plan your query step-by-step inside the <scratchpad></scratchpad> tags",
            },
            {"role": "user", "content": f"{question}"},
        ]
    return client.chat.completions.create(
        model="adrienbrault/nous-hermes2pro:Q4_0",
        temperature=0.1,
        response_model=SelfReflection,
        messages=messages,
    ), messages

def self_reflection_loop(question: str) -> QuestionAnswer:
    response, messages = ask_ai(question)
    
    agent_calls = ""
    for query in response.sub_questions:
        agent_calls += f"<agent>\n{query}\n</agent>\n"

    content = f"<scratchpad> {response.scratchpad} </scratchpad>\n"
    content += agent_calls
    messages.append({
        "role": "assistant",
        "content": content
    })

    context = ""

    with ThreadPoolExecutor() as executor:
        futures = []

        for query in response.sub_questions:
            future = executor.submit(ask_agent, query, context)
            futures.append(future)

        agent_messages = [] 
        for i in range(len(futures)):
            result = futures[i].result()

            agent_messages.append(
                {
                    "role": f"agent-{i}",
                    "content": response.sub_questions[i]
                }
            )
            agent_messages.append(
                {
                    "role": f"agent-{i}-response",
                    "content": result.answer
                }
            )
            context += f"<agent id={i}>\n"
            context += f"sub_query: {response.sub_questions[i]}\n"
            context += f"answer: {result.answer}\n"
            context += f"</agent>\n"
    print(context)
    messages.append({
        "role": "agents",
        "content": agent_messages
    })
    final_answer = ask_agent(question, context)
    messages.append({
        "role": "assistant",
        "content": final_answer.answer
    })
    return final_answer, messages

In [59]:
question = "Who is Jay Gatsby?"
response, messages = self_reflection_loop(question)

<agent id=0>
sub_query: In which book or story can Jay Gatsby be found?
answer: Jay Gatsby can be found in the book 'The Great Gatsby'.
</agent>
<agent id=1>
sub_query: Who is the author of the work featuring Jay Gatsby?
answer: The author of the work featuring Jay Gatsby is F. Scott Fitzgerald.
</agent>
<agent id=2>
sub_query: What is the full name and background of Jay Gatsby?
answer: Jay Gatsby, full name James Gatz, was a fictional character in F. Scott Fitzgerald's 1925 novel, The Great Gatsby. He came from an impoverished background and gained wealth through his involvement in the underworld during World War I.
</agent>
<agent id=3>
sub_query: What is the main character's relationship with Jay Gatsby?
answer: The main character, Nick Carraway, is a friend and narrator of the story. He lives next door to Jay Gatsby and becomes closely acquainted with him throughout the novel.
</agent>
<agent id=4>
sub_query: How does Jay Gatsby's past influence his actions in the story?
answer: Ja

In [60]:
response.answer

"Jay Gatsby is a fictional character in F. Scott Fitzgerald's 1925 novel, The Great Gatsby. He comes from an impoverished background and gained wealth through his involvement in the underworld during World War I."

In [61]:
print(messages)

[{'role': 'system', 'content': "You are a world class agentic framework to decompose a given question into 5 sub-questions to help collect evidences for final answer. You must plan your query step-by-step inside the <scratchpad></scratchpad> tags\n\n\nAs a genius expert, your task is to understand the content and provide\nthe parsed objects in json that match the following json_schema:\n\n\n{'properties': {'scratchpad': {'title': 'Scratchpad', 'type': 'string'}, 'sub_questions': {'items': {'type': 'string'}, 'title': 'Sub Questions', 'type': 'array'}}, 'required': ['scratchpad', 'sub_questions'], 'title': 'SelfReflection', 'type': 'object'}\n\nMake sure to return an instance of the JSON, not the schema itself\n"}, {'role': 'user', 'content': 'Who is Jay Gatsby?'}, {'role': 'assistant', 'content': "<scratchpad> Who is Jay Gatsby? </scratchpad>\n<agent>\nIn which book or story can Jay Gatsby be found?\n</agent>\n<agent>\nWho is the author of the work featuring Jay Gatsby?\n</agent>\n<age

In [63]:
print(json.dumps(messages, indent=2))

[
  {
    "role": "system",
    "content": "You are a world class agentic framework to decompose a given question into 5 sub-questions to help collect evidences for final answer. You must plan your query step-by-step inside the <scratchpad></scratchpad> tags\n\n\nAs a genius expert, your task is to understand the content and provide\nthe parsed objects in json that match the following json_schema:\n\n\n{'properties': {'scratchpad': {'title': 'Scratchpad', 'type': 'string'}, 'sub_questions': {'items': {'type': 'string'}, 'title': 'Sub Questions', 'type': 'array'}}, 'required': ['scratchpad', 'sub_questions'], 'title': 'SelfReflection', 'type': 'object'}\n\nMake sure to return an instance of the JSON, not the schema itself\n"
  },
  {
    "role": "user",
    "content": "Who is Jay Gatsby?"
  },
  {
    "role": "assistant",
    "content": "<scratchpad> Who is Jay Gatsby? </scratchpad>\n<agent>\nIn which book or story can Jay Gatsby be found?\n</agent>\n<agent>\nWho is the author of the wo